## Генерация видео путем интерполяции в семантическом пространстве Stable Diffusion

В данном ноутбуке мы учимся создавать видеофрагменты за счет интерполяции текстовых семантических векторов в Stable Diffusion. Просьба при использовании этого ноутбука ссылаться на первоисточник: http://eazify.net/sdvideo.

Для начала, устанавливаем необходимые библиотеки:

In [1]:
!pip install -q diffusers transformers scipy ftfy accelerate
!pip install -q "ipywidgets>=7,<8"
!pip install -q xformers
!pip install -q triton==2.0.0.dev20221120

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 737.4/737.4 KB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 KB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.8/212.8 KB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.7/18.7 MB 10.6 MB/s eta 0:00:00


Далее, вводим свой токен Huggingface для работы с моделями:

In [2]:
from google.colab import output
from huggingface_hub import notebook_login
output.enable_custom_widget_manager()
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


Теперь загружаем модель Stable Diffusion:

In [3]:
import torch
from diffusers import StableDiffusionPipeline

#model_name = "CompVis/stable-diffusion-v1-4"
#model_name = "stabilityai/stable-diffusion-2-1"
model_name = "runwayml/stable-diffusion-v1-5"

pipe = StableDiffusionPipeline.from_pretrained(model_name, revision="fp16", torch_dtype=torch.float16, use_auth_token=True, safety_checker=None)  
pipe.enable_xformers_memory_efficient_attention()
pipe = pipe.to("cuda")

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

/usr/local/lib/python3.9/dist-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(
You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


Определяем несколько вспомогательных функций:

In [7]:
import matplotlib.pyplot as plt

def display_images(l):
    n=len(l)
    fig,ax = plt.subplots(1,n)
    for i,im in enumerate(l):
        ax[i].imshow(im)
        ax[i].axis('off')
    fig.set_size_inches(fig.get_size_inches()*n)
    plt.tight_layout()
    plt.show()
    

Для генерации видео мы задаём начальный и конечный запрос, а также количество кадров и параметры генерации:


In [6]:
start = "Oil painting of a nice happy couple waliking hand in hand by the sea shore on a sunny day"
finish = "Oil painting of a nice sad couple walking hand in hand under pouring rain in the big city in autumn"
frames = 90

params = {
    "width" : 512,
    "height" : 768,
    "num_inference_steps" : 20
}

Опишем несколько вспомогательный функций:

In [8]:
def get_prompt_embeds(prompt):
  device = pipe.text_encoder.device
  text_inputs = pipe.tokenizer(
    prompt, padding="max_length",
    max_length=pipe.tokenizer.model_max_length,
    truncation=True,
    return_tensors="pt")
  
  text_input_ids = text_inputs.input_ids
  untruncated_ids = pipe.tokenizer(prompt, padding="longest", return_tensors="pt").input_ids

  if untruncated_ids.shape[-1] >= text_input_ids.shape[-1] and not torch.equal(
      text_input_ids, untruncated_ids):
    removed_text = pipe.tokenizer.batch_decode(
        untruncated_ids[:, pipe.tokenizer.model_max_length - 1 : -1])
    print(
        "The following part of your input was truncated because CLIP can only handle sequences up to"
        f" {pipe.tokenizer.model_max_length} tokens: {removed_text}"
    )

  if hasattr(pipe.text_encoder.config, "use_attention_mask") and pipe.text_encoder.config.use_attention_mask:
      attention_mask = text_inputs.attention_mask.to(pipe.device)
  else:
      attention_mask = None

  prompt_embeds = pipe.text_encoder(
      text_input_ids.to(device),
      attention_mask=attention_mask,
  )
  prompt_embeds = prompt_embeds[0]
  return prompt_embeds

vstart = get_prompt_embeds(start)
vfinish = get_prompt_embeds(finish)

In [9]:
shape = (1, pipe.unet.in_channels, params['height'] // pipe.vae_scale_factor, params['width'] // pipe.vae_scale_factor)
latents = torch.randn(shape,dtype=pipe.unet.dtype)

Следующая ячейка непосредственно генерирует все изображения. Это может занять длительное время, в зависимости от количества кадров.

In [10]:
batch_size = 3
vec = (vfinish[0]-vstart[0])/frames
encs = torch.stack([ vstart[0]+vec*i for i in range(frames)])
lats = torch.stack([latents[0]]*batch_size)
res = []
for b in torch.split(encs,batch_size):
  r = pipe(prompt_embeds=b,latents=lats,**params).images
  res.extend(r)

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

Записываем отдельные кадры на диск:

In [11]:
!rm frame*.jpg
for i,im in enumerate(res):
  im.save(f'frame_{i:03d}.jpg')

rm: cannot remove 'frame*.jpg': No such file or directory


И собираем все кадры вместе с помощью `ffmpeg`:

In [13]:
!rm *.mp4
!ffmpeg -r 15 -i frame_%03d.jpg -c:v libx264 -vf fps=25 -pix_fmt yuv420p out.mp4

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

### Генерация более длинного видео

Для этого мы описываем последовательность фраз, и генерируем подряд последовательность переходов между ними.

In [17]:
prompts = """
Oil painting of a nice young happy couple waliking hand in hand by the sea shore on a sunny day, with ships in the distance
Oil painting of a young couple sleeping by the sea shore on a dark starry night, with glowing ships in the distance
Oil painting of a nice sad couple walking hand in hand under pouring rain in the big city in autumn on a rainy day
Oil painting of a couple sleeping in their bed at night, with lights from street lamps coming through the window
Oil painting of a nice old couple walking hand in hand in the snow in the rural village on a sunny day
Oil painting of an old couple sitting in front of the TV in the evening, nice rural interiour
Oil painting of an old couple going to bed at night in the rural village house
Oil painting of a starry night with full moon, scary atmosphere
"""

frames = 30 # per episode
params = {
    "width" : 512,
    "height" : 512,
    "num_inference_steps" : 30
}


In [18]:
prompts = [x for x in prompts.split('\n') if len(x)>1]
vecs = [get_prompt_embeds(x)[0] for x in prompts]

In [ ]:
res = []
for v1,v2 in zip(vecs[:-1],vecs[1:]):
  print(f"NEW Episode")
  vec = (v2-v1)/frames
  encs = torch.stack([ v1+vec*i for i in range(frames)])
  lats = torch.stack([latents[0]]*batch_size)
  for b in torch.split(encs,batch_size):
    r = pipe(prompt_embeds=b,latents=lats,**params).images
    res.extend(r) 

NEW Episode


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

In [ ]:
!rm frame*.jpg
for i,im in enumerate(res):
  im.save(f'frame_{i:03d}.jpg')

In [ ]:
!rm *.mp4
!ffmpeg -r 5 -i frame_%03d.jpg -c:v libx264 -vf fps=25 -pix_fmt yuv420p out.mp4

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e